In [1]:
import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
import nltk.corpus
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import datasets, linear_model
import scipy.sparse as sps
from scipy.sparse import coo_matrix, hstack, vstack, csr_matrix
from scipy import io
from datetime import datetime
import gc
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer
%matplotlib inline
from IPython.display import display, HTML
from pprint import pprint

In [107]:
FE.ReadCsv('train')

,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,Demanda_uni_equil
0,3,1111,1,4402,50789,32846,2,10.920000,0,0,2
1,3,1112,1,2103,1209569,36748,4,46.119999,0,0,4


Semana                 uint8
Agencia_ID            uint16
Canal_ID               uint8
Ruta_SAK              uint16
Cliente_ID            uint32
Producto_ID           uint16
Venta_uni_hoy         uint16
Venta_hoy            float32
Dev_uni_proxima       uint32
Dev_proxima          float32
Demanda_uni_equil     uint32
dtype: object
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1007 entries, 0 to 1006
Data columns (total 11 columns):
Semana               1007 non-null uint8
Agencia_ID           1007 non-null uint16
Canal_ID             1007 non-null uint8
Ruta_SAK             1007 non-null uint16
Cliente_ID           1007 non-null uint32
Producto_ID          1007 non-null uint16
Venta_uni_hoy        1007 non-null uint16
Venta_hoy            1007 non-null float32
Dev_uni_proxima      1007 non-null uint32
Dev_proxima          1007 non-null float32
Demanda_uni_equil    1007 non-null uint32
dtypes: float32(2), uint16(4), uint32(3), uint8(2)
memory usage: 37.4 KB
None


In [118]:
groupCount = FE.train.loc[:,['Semana','Producto_ID','Cliente_ID']+['Demanda_uni_equil']].groupby(['Semana','Producto_ID','Cliente_ID']).count()
groupCount[groupCount['Demanda_uni_equil']>1]

Demanda_uni_equil
Semana Producto_ID Cliente_ID                   
6      1240        196327                      2
       1278        200910                      2
       1284        1270889                     2
       3144        897395                      2
7      1212        4441528                     2
9      5000        4457462                     2
       48577       979169                      2

In [108]:
investigate = JoinWithAggregate(FE.train,['Semana','Producto_ID','Cliente_ID'])
investigate

Index([u'count'], dtype='object')


,count,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,Demanda_uni_equil
454,2,6,1330,1,1155,196327,1240,6,50.279999,0,0.00,6
462,2,6,1330,1,1155,196327,1240,6,50.279999,0,0.00,6
427,2,6,1121,1,1401,200910,1278,30,135.000000,0,0.00,30
456,2,6,1121,1,1401,200910,1278,30,135.000000,0,0.00,30
441,2,6,1223,1,1168,1270889,1284,5,15.100000,0,0.00,5
442,2,6,1223,1,1168,1270889,1284,5,15.100000,0,0.00,5
455,2,6,1912,1,1006,897395,3144,1,21.320000,1,21.32,0
511,2,6,1912,1,1006,897395,3144,1,21.320000,1,21.32,0
667,2,7,2218,1,1206,4441528,1212,2,18.139999,0,0.00,2
674,2,7,2218,1,1206,4441528,1212,2,18.139999,0,0.00,2


In [106]:
#investigate.loc[:,'Demanda_uni_equil'] =  investigate.loc[:,'Demanda_uni_equil'].apply(np.log1p).astype('uint16')
#investigate

,count,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,Demanda_uni_equil
454,2,6,1330,1,1155,196327,1240,6,50.279999,0,0.00,0
462,2,6,1330,1,1155,196327,1240,6,50.279999,0,0.00,0
427,2,6,1121,1,1401,200910,1278,30,135.000000,0,0.00,0
456,2,6,1121,1,1401,200910,1278,30,135.000000,0,0.00,0
441,2,6,1223,1,1168,1270889,1284,5,15.100000,0,0.00,0
442,2,6,1223,1,1168,1270889,1284,5,15.100000,0,0.00,0
455,2,6,1912,1,1006,897395,3144,1,21.320000,1,21.32,0
511,2,6,1912,1,1006,897395,3144,1,21.320000,1,21.32,0
667,2,7,2218,1,1206,4441528,1212,2,18.139999,0,0.00,5
674,2,7,2218,1,1206,4441528,1212,2,18.139999,0,0.00,5


In [102]:
def JoinWithAggregate(df, columnList):
    filtered = df.loc[:,columnList+['Demanda_uni_equil']].groupby(columnList).count()
    filtered = filtered[filtered.loc[:,'Demanda_uni_equil']>1]
    filtered.columns = ['count']
    print filtered.columns
    gc.collect()
    return filtered.merge(FE.train,left_index = True, right_on= columnList, how ="left")

In [7]:
class FeatureEngineering:
    #ValidationStart = 0
    #lag = 0
    #trainHdfPath = ""
    #trainHdfFile = ""
    #testHdfPath = ""
    #testHdfFile = ""
    #train = pd.DataFrame()
    #test = pd.DataFrame()
    
    def __init__(self, ValidationStart, ValidationSmallerThan, trainHdfPath, trainHdfFile, testHdfPath, testHdfFile, 
                 testTypes, trainTypes, trainCsvPath, testCsvPath, maxLag=0):
        self.ValidationStart = ValidationStart
        self.ValidationSmallerThan = ValidationSmallerThan
        self.maxLag = maxLag
        self.trainHdfPath = trainHdfPath
        self.trainHdfFile = trainHdfFile
        self.testHdfPath = testHdfPath
        self.testHdfFile = testHdfFile
        self.testTypes = testTypes
        self.trainTypes = trainTypes
        self.trainCsvPath = trainCsvPath
        self.testCsvPath = testCsvPath
        
    @staticmethod
    def __printDataFrameBasics__(data):
        display(data.head(2))
        print data.dtypes
        gc.collect()
        print(data.info(memory_usage=True))
        
    @staticmethod    
    def __changeIndexTypeToLowerMemory(data):
        ##########
        #This is very critical, i accept max number is 2^32. Also, if don't do that, memory gets so much higher..
        ##########
        data.index = data.index.astype('uint32')
        
    def ReadHdf(self, trainOrTestOrBoth):
        '''Reads and holds Df in object memory'''
        if trainOrTestOrBoth == '':
            raise "qweqweq"
            
        if trainOrTestOrBoth == 'train' or trainOrTestOrBoth=='both':
            self.train = pd.read_hdf(self.trainHdfPath,self.trainHdfFile)
            FeatureEngineering.__changeIndexTypeToLowerMemory(self.train)
            FeatureEngineering.__printDataFrameBasics__(self.train)
        if trainOrTestOrBoth == 'test' or trainOrTestOrBoth=='both':
            self.test = pd.read_hdf(self.testHdfPath,self.testHdfFile)
            FeatureEngineering.__changeIndexTypeToLowerMemory(self.test)
            FeatureEngineering.__printDataFrameBasics__(self.test)
        
    def ReadCsv(self, trainOrTestOrBoth):
        '''Reads and holds Df in object memory'''
        if trainOrTestOrBoth == '':
            raise "qweqweq"  
        if trainOrTestOrBoth == 'train' or trainOrTestOrBoth=='both':
            self.train = pd.read_csv(self.trainCsvPath, usecols=self.trainTypes.keys(), dtype=self.trainTypes)
            #__changeIndexTypeToLowerMemory(self.train)
            FeatureEngineering.__printDataFrameBasics__(self.train)
        if trainOrTestOrBoth == 'test' or trainOrTestOrBoth=='both':
            self.test = pd.read_csv(self.testCsvPath, usecols=self.testTypes.keys(), dtype=self.testTypes)
            #__changeIndexTypeToLowerMemory(self.test)
            FeatureEngineering.__printDataFrameBasics__(self.test)

    @staticmethod
    def ConvertCsvToHdf(csvPath, HdfPath, HdfName, ColumnTypeDict ):
        tempDf = pd.read_csv(csvPath, usecols=ColumnTypeDict.keys(), dtype=ColumnTypeDict)
        tempDf.to_hdf(HdfPath, HdfName, format='t')
        print "ConvertCsvToHdf is done.."

    def Preprocess(self, trainOrTestOrBoth, columnFunctionTypeList):
        '''columnFunctionTypeList = [ ['C1',Func1,Type], ['C2',Func2,Type],..    ]'''
        for column, func, localType in columnFunctionTypeList:
            if trainOrTestOrBoth == 'train' or trainOrTestOrBoth=='both':
                self.train.loc[:,column] =  self.train[column].apply(func).astype(localType)
            elif trainOrTestOrBoth == 'test' or trainOrTestOrBoth=='both':
                self.test.loc[:,column] =  self.test[column].apply(func).astype(localType)
            
    def SaveDataFrameToHdf(self,trainOrTestOrBoth):
        if trainOrTestOrBoth == 'train' or trainOrTestOrBoth=='both':
            self.train.to_hdf(self.trainHdfPath, self.trainHdfFile, format='t')
        if trainOrTestOrBoth == 'test' or trainOrTestOrBoth=='both':
            self.test.to_hdf(self.testHdfPath, self.testHdfFile, format='t')
        
    def AddDemandaGeneralMean(self):
        DemandaMeanWithoutLag = self.train.loc[self.train.loc[:,'Semana'] < self.ValidationStart]
        self.train.loc[:,"DemandaGeneralMean"] = DemandaMeanWithoutLag['Demanda_uni_equil'].mean()
        self.train.loc[:,"DemandaGeneralMean"] = self.train.loc[:,"DemandaGeneralMean"].astype('float32')
        display(self.train)
        
    def AddConfigurableFeaturesToTrain(self, config):
        if config.lag > self.maxLag:
            self.maxLag = config.lag
        
        lag = 0 if config.lag == 0 else config.lag-1
        tempData = self.train.loc[self.train.loc[:,'Semana'] < self.ValidationStart - lag]
        #display(tempData)
        if(config.lag != 0):
            tempData.loc[:,'Semana'] = tempData.loc[:,'Semana'].apply(lambda x:x + config.lag)
        #display(tempData)
        
        #Means iterative.. eliminate as long as np.nan exists..If there is already one, don't create, use the existing
        if config.targetVariable != "" and  config.targetVariable not in self.train.columns:
            self.train.loc[:,config.targetVariable] = np.nan
        
        for name,groups in config.nameAndGroups:
            if name not in self.train.columns:
                print "{} is not in columns..".format(name)
                
                ##First, select columns not to fill memory..then groupby..
                
                groupedDataframe =  pd.DataFrame({name:tempData[groups+['Demanda_uni_equil']].groupby(groups)
                    ['Demanda_uni_equil'].mean().astype('float32')})
                #join..
                
                display(groupedDataframe)
                self.train = self.train.merge( groupedDataframe, left_on=groups,
                                  right_index=True, how='left', sort=False,copy=False)
            else:
                print "{} is in columns..".format(name)
            
            display(self.train)
            
            #Means iterative..!!!!!
            if config.targetVariable != "":
                self.train.loc[pd.isnull(self.train[config.targetVariable]), 
                    config.targetVariable] = self.train.loc[pd.isnull(self.train[config.targetVariable]), name]
                count = self.train.loc[self.train['Semana'] >= self.ValidationStart,config.targetVariable].isnull().sum()
                print "Count of missing numbers after {} in validation part in column {} is {}".format(name, 
                    config.targetVariable,str(count))
                #display(self.train)
                #If column is already in Dataframe and we want to fill target variable, this deletes columns!!!
                if(config.deleteColumns):
                    self.train.drop(name, axis=1, inplace=True)
                gc.collect()
                if count == 0:
                    break
        display(self.train)        
        gc.collect()
        return 
    def DeleteLaggedWeeksFromTrain(self):
        self.train = self.train.loc[self.train.loc[:,'Semana']>= 3 + self.maxLag]
        display(self.train)

In [8]:
parameterDict =       {"ValidationStart":6, 
 "ValidationSmallerThan":7,
   "maxLag":2,
    "trainHdfPath":'../../input/train_1000.h5',
    "trainHdfFile":"train",
    "testHdfPath":"../../input/test_1000.h5",
    "testHdfFile":"test", 
    "trainTypes" : {'Semana':np.uint8, 'Agencia_ID':np.uint16, 'Canal_ID':np.uint8,'Ruta_SAK':np.uint16, 
        'Cliente_ID':np.uint32, 'Producto_ID':np.uint16,'Venta_uni_hoy':np.uint16, 'Venta_hoy':np.float32,
                    'Dev_uni_proxima': np.uint32, 'Dev_proxima':np.float32,'Demanda_uni_equil':np.uint32}, 
    "testTypes" : {'id':np.uint32,'Semana':np.uint8, 'Agencia_ID':np.uint16, 'Canal_ID':np.uint8,'Ruta_SAK':np.uint16,
        'Cliente_ID':np.uint32, 'Producto_ID':np.uint16},
    "trainCsvPath":'../../input/train_1000.csv'   ,
    "testCsvPath":'../../input/test_1000.csv'}

FE = FeatureEngineering(**parameterDict)
print FE.__dict__

{'maxLag': 2, 'testTypes': {'Cliente_ID': <type 'numpy.uint32'>, 'Ruta_SAK': <type 'numpy.uint16'>, 'Canal_ID': <type 'numpy.uint8'>, 'Producto_ID': <type 'numpy.uint16'>, 'Agencia_ID': <type 'numpy.uint16'>, 'Semana': <type 'numpy.uint8'>, 'id': <type 'numpy.uint32'>}, 'testHdfFile': 'test', 'trainTypes': {'Dev_proxima': <type 'numpy.float32'>, 'Venta_uni_hoy': <type 'numpy.uint16'>, 'Cliente_ID': <type 'numpy.uint32'>, 'Demanda_uni_equil': <type 'numpy.uint32'>, 'Ruta_SAK': <type 'numpy.uint16'>, 'Canal_ID': <type 'numpy.uint8'>, 'Venta_hoy': <type 'numpy.float32'>, 'Producto_ID': <type 'numpy.uint16'>, 'Agencia_ID': <type 'numpy.uint16'>, 'Dev_uni_proxima': <type 'numpy.uint32'>, 'Semana': <type 'numpy.uint8'>}, 'trainCsvPath': '../../input/train_1000.csv', 'testHdfPath': '../../input/test_1000.h5', 'testCsvPath': '../../input/test_1000.csv', 'ValidationStart': 6, 'trainHdfFile': 'train', 'trainHdfPath': '../../input/train_1000.h5', 'ValidationSmallerThan': 7}
